# Lucid Sonic Dreams - Music Visualization with GANs

Generate stunning AI music visualizations using StyleGAN3 and R3GAN.

**Supported Models:**
- **StyleGAN3** - High quality, many pretrained styles
- **R3GAN** - Modern GAN baseline with SOTA FID scores (NeurIPS 2024)

**Performance Features:**
- FP16/BFloat16 inference (2x faster on GPU)
- torch.compile() support (PyTorch 2.0+)
- Custom style models (.pkl files)
- Custom effects (zoom, swirl, etc.)
- **uv package manager** (~10x faster installs)

**Requirements:**
- GPU runtime (Runtime > Change runtime type > T4 GPU)
- Audio file (.mp3, .wav)
- Python 3.12+

## How It Works

There are **over 30 parameters** you can tune, offering tons of flexibility. Here's a basic understanding:

### The Process

1. First, a batch of input vectors corresponding to output images is initialized. Linear interpolations between these vectors are produced, serving as the "base" vectors.
2. Three components react to the audio: **Pulse**, **Motion**, and **Class**. These modify the "base" vectors accordingly.

   * **Pulse** - How the visuals "pulse" to the beat. Reacts to percussive elements by default.
   * **Motion** - How the visuals are "pushed forward" by the music. Reacts to harmonic elements by default.
   * **Class** - Object labels in generated images (e.g., Van Gogh, Warhol for WikiArt). Reacts to audio pitch.

3. Finally, additional effects (contrast, flash, custom) are applied.

### Key Parameters to Start With

Start by tuning these, then explore others:
- **speed_fpm** - Frames Per Minute, controls morphing speed
- **pulse_react** - Strength of pulse effect
- **motion_react** - Strength of motion effect
- **class_pitch_react** - How much pitch affects class changes

## 1. Setup & Installation

Run this cell to install all dependencies using uv (much faster than pip!).

In [ ]:
# Check GPU availability
!nvidia-smi

# Install uv (fast Python package manager)
!curl -LsSf https://astral.sh/uv/install.sh | sh
import os
os.environ["PATH"] = f"/root/.local/bin:{os.environ['PATH']}"

# Clone lucid-sonic-dreams
!git clone https://github.com/RayceRossum/lucid-sonic-dreams.git /content/lucid-sonic-dreams
%cd /content/lucid-sonic-dreams

# Install with uv (much faster than pip, ~10x speedup)
!uv pip install --system -e .
!uv pip install --system ninja  # For faster StyleGAN compilation

# Setup StyleGAN3
!git clone https://github.com/NVlabs/stylegan3 /content/lucid-sonic-dreams/stylegan3
!ln -s /content/lucid-sonic-dreams/stylegan3 /content/lucid-sonic-dreams/stylegan2

print("\n" + "="*50)
print("Installation complete!")
print("="*50)

## 2. Mount Google Drive

Mount to access your audio files, custom models, and save outputs.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Create output directory
import os
OUTPUT_DIR = '/content/drive/MyDrive/LucidSonicDreams/outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory: {OUTPUT_DIR}")

## 3. Check System & Available Styles

In [ ]:
import torch
from lucidsonicdreams import (
    show_styles, show_r3gan_styles, is_colab, suggest_batch_size,
    get_gpu_memory_mb, get_free_gpu_memory_mb
)

print("=" * 50)
print("SYSTEM INFO")
print("=" * 50)
print(f"Running on Colab: {is_colab()}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {get_gpu_memory_mb():.0f} MB total")
    print(f"GPU Memory Free: {get_free_gpu_memory_mb():.0f} MB")
print(f"torch.compile available: {hasattr(torch, 'compile')}")

print("\n" + "=" * 50)
print("RECOMMENDED SETTINGS")
print("=" * 50)
print(f"Batch size for 512px: {suggest_batch_size(512)}")
print(f"Batch size for 1024px: {suggest_batch_size(1024)}")

print("\n" + "=" * 50)
print("R3GAN MODELS (Modern GAN Baseline)")
print("=" * 50)
show_r3gan_styles()

print("\n" + "=" * 50)
print("STYLEGAN STYLES")
print("=" * 50)
show_styles()

## 4. Configure Your Visualization (Interactive)

Use the dropdowns below to configure your visualization settings. After running this cell, modify the values and re-run the generation cell.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# ============================================================
# Define available styles
# ============================================================
STYLES = {
    # StyleGAN styles
    "abstract": "abstract art",
    "modern": "modern art",
    "textures": "textures",
    "microscope": "microscope images",
    "landscapes": "landscape photos",
    "faces": "photos of faces",
    
    # R3GAN models (Modern GAN - NeurIPS 2024)
    "r3gan_faces_256": "r3gan_ffhq_256",      # 256x256 faces (FID: 2.75)
    "r3gan_faces_64": "r3gan_ffhq_64",        # 64x64 faces, fast (FID: 1.95)
    "r3gan_cifar": "r3gan_cifar10",           # 32x32, 10 classes (FID: 1.96)
    "r3gan_imagenet_64": "r3gan_imagenet_64", # 64x64, 1000 classes (FID: 2.09)
    "r3gan_imagenet_32": "r3gan_imagenet_32", # 32x32, 1000 classes (FID: 1.27)
    
    # Add custom .pkl paths below:
    # "custom_wikiart": "/content/drive/MyDrive/models/wikiart.pkl",
}

# ============================================================
# Create interactive widgets
# ============================================================

# Style selection
style_dropdown = widgets.Dropdown(
    options=list(STYLES.keys()),
    value='abstract',
    description='Style:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

# Resolution
resolution_dropdown = widgets.Dropdown(
    options=[256, 512, 1024],
    value=512,
    description='Resolution:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

# FPS
fps_dropdown = widgets.Dropdown(
    options=[24, 30, 60],
    value=30,
    description='FPS:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

# Duration
duration_slider = widgets.IntSlider(
    value=30,
    min=5,
    max=300,
    step=5,
    description='Duration (sec):',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

# Speed FPM
speed_slider = widgets.FloatSlider(
    value=12,
    min=0,
    max=30,
    step=1,
    description='Speed (FPM):',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

# Motion react
motion_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=2,
    step=0.1,
    description='Motion React:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

# Pulse react
pulse_slider = widgets.FloatSlider(
    value=0.5,
    min=0,
    max=2,
    step=0.1,
    description='Pulse React:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

# Truncation
truncation_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=1.5,
    step=0.1,
    description='Truncation:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

# Effects toggles
contrast_checkbox = widgets.Checkbox(value=False, description='Contrast Effect')
flash_checkbox = widgets.Checkbox(value=False, description='Flash Effect')

# Effect strengths (shown when enabled)
contrast_strength = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Contrast:')
flash_strength = widgets.FloatSlider(value=0.3, min=0, max=1, step=0.1, description='Flash:')

# Audio reactivity presets
preset_dropdown = widgets.Dropdown(
    options=[
        ('Custom', 'custom'),
        ('Calm/Ambient', 'calm'),
        ('Balanced', 'balanced'),
        ('Energetic/EDM', 'energetic'),
        ('Experimental', 'experimental'),
    ],
    value='balanced',
    description='Preset:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

def on_preset_change(change):
    """Apply preset values when preset changes."""
    preset = change['new']
    if preset == 'calm':
        speed_slider.value = 4
        motion_slider.value = 0.2
        pulse_slider.value = 0.2
        truncation_slider.value = 0.8
    elif preset == 'balanced':
        speed_slider.value = 12
        motion_slider.value = 0.5
        pulse_slider.value = 0.5
        truncation_slider.value = 1.0
    elif preset == 'energetic':
        speed_slider.value = 24
        motion_slider.value = 1.0
        pulse_slider.value = 1.2
        truncation_slider.value = 1.0
        contrast_checkbox.value = True
        flash_checkbox.value = True
    elif preset == 'experimental':
        speed_slider.value = 6
        motion_slider.value = 1.5
        pulse_slider.value = 1.8
        truncation_slider.value = 1.2
        contrast_checkbox.value = True
        flash_checkbox.value = True

preset_dropdown.observe(on_preset_change, names='value')

# Layout with sections
video_settings = widgets.VBox([
    widgets.HTML('<h4>📹 Video Settings</h4>'),
    resolution_dropdown,
    fps_dropdown,
    duration_slider,
])

style_settings = widgets.VBox([
    widgets.HTML('<h4>🎨 Style Settings</h4>'),
    style_dropdown,
    truncation_slider,
])

audio_settings = widgets.VBox([
    widgets.HTML('<h4>🎵 Audio Reactivity</h4>'),
    preset_dropdown,
    speed_slider,
    motion_slider,
    pulse_slider,
])

effects_settings = widgets.VBox([
    widgets.HTML('<h4>✨ Effects</h4>'),
    widgets.HBox([contrast_checkbox, flash_checkbox]),
    contrast_strength,
    flash_strength,
])

# Display all widgets
display(widgets.VBox([
    widgets.HTML('<h3>🎬 Lucid Sonic Dreams Configuration</h3>'),
    widgets.HBox([
        widgets.VBox([video_settings, style_settings]),
        widgets.VBox([audio_settings, effects_settings]),
    ]),
    widgets.HTML('<p><i>Tip: Select a preset to auto-configure settings, then fine-tune as needed.</i></p>')
]))

# Store current config for easy access
def get_config():
    """Get current configuration from widgets."""
    return {
        'style': STYLES[style_dropdown.value],
        'style_name': style_dropdown.value,
        'resolution': resolution_dropdown.value,
        'fps': fps_dropdown.value,
        'duration': duration_slider.value,
        'speed_fpm': speed_slider.value,
        'motion_react': motion_slider.value,
        'pulse_react': pulse_slider.value,
        'truncation': truncation_slider.value,
        'contrast_strength': contrast_strength.value if contrast_checkbox.value else None,
        'flash_strength': flash_strength.value if flash_checkbox.value else None,
    }

print("\n✅ Configuration widgets loaded! Adjust settings above, then run the generation cell.")

## 5. Select Audio Files

Option A: Upload from your computer  
Option B: Use files from Google Drive

In [ ]:
# Option A: Upload audio file
from google.colab import files

print("Upload your audio file (.mp3 or .wav):")
uploaded = files.upload()
AUDIO_FILE = list(uploaded.keys())[0]
print(f"\nUploaded: {AUDIO_FILE}")

In [ ]:
# Option B: Use files from Google Drive folder
import glob
import os

AUDIO_FOLDER = "/content/drive/MyDrive/LucidSonicDreams/audio/"  # Edit this path

# Find all audio files
audio_files = glob.glob(os.path.join(AUDIO_FOLDER, "*.mp3")) + \
              glob.glob(os.path.join(AUDIO_FOLDER, "*.wav"))

print(f"Found {len(audio_files)} audio files:")
for i, f in enumerate(audio_files):
    print(f"  [{i}] {os.path.basename(f)}")

# Select file by index
AUDIO_INDEX = 0  # Change this to select different file
AUDIO_FILE = audio_files[AUDIO_INDEX] if audio_files else None
print(f"\nSelected: {AUDIO_FILE}")

## 6. Generate Video

Run this cell to generate your video using the settings from the dropdowns above.

In [ ]:
from lucidsonicdreams import LucidSonicDream
import os

# Get configuration from widgets
config = get_config()

print("=" * 50)
print("GENERATION CONFIG")
print("=" * 50)
for k, v in config.items():
    print(f"  {k}: {v}")
print("=" * 50)

# Create visualizer
L = LucidSonicDream(
    song=AUDIO_FILE,
    style=config['style']
)

# Generate video
OUTPUT_FILE = f"{os.path.splitext(os.path.basename(AUDIO_FILE))[0]}_{config['style_name']}.mp4"

L.hallucinate(
    file_name=OUTPUT_FILE,
    resolution=config['resolution'],
    fps=config['fps'],
    duration=config['duration'],
    batch_size=1,
    speed_fpm=config['speed_fpm'],
    motion_react=config['motion_react'],
    pulse_react=config['pulse_react'],
    truncation=config['truncation'],
    contrast_strength=config['contrast_strength'],
    flash_strength=config['flash_strength'],
)

print(f"\n✅ Video saved: {OUTPUT_FILE}")

## 7. Preview & Download

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# Display video
mp4 = open(OUTPUT_FILE, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=512 controls><source src="{data_url}" type="video/mp4"></video>')

In [ ]:
# Download to your computer
files.download(OUTPUT_FILE)

# Or copy to Google Drive
import shutil
shutil.copy(OUTPUT_FILE, OUTPUT_DIR)
print(f"Copied to: {OUTPUT_DIR}/{OUTPUT_FILE}")

## 8. Full Quality Render with All Parameters

### Parameter Reference

#### Initialization
- **speed_fpm** (Default: 12) - Frames Per Minute. More = faster morphing. 0 = single reactive image.

#### Pulse Parameters
- **pulse_react** (Default: 0.5) - Strength of pulse. Keep between 0-2.
- **pulse_percussive** (Default: True) - React to percussive elements.
- **pulse_harmonic** (Default: False) - React to harmonic elements.
- **pulse_audio** - Path to isolated audio track for pulse.

#### Motion Parameters
- **motion_react** (Default: 0.5) - Strength of motion. Keep between 0-2.
- **motion_percussive** (Default: False) - React to percussive elements.
- **motion_harmonic** (Default: True) - React to harmonic elements.
- **motion_randomness** (Default: 0.5) - Randomness of motion. 0-1.
- **truncation** (Default: 1) - Visual variety. Lower = less variety. 0-1.

#### Class Parameters
- **classes** - List of up to 12 numerical labels.
- **dominant_classes_first** (Default: False) - Sort by prominence.
- **class_pitch_react** (Default: 0.5) - Strength of pitch reaction. 0-2.
- **class_smooth_seconds** (Default: 1) - Interpolation smoothness.
- **class_complexity** (Default: 1) - Image complexity. 0-1.
- **class_shuffle_seconds** - When to reshuffle class mapping.
- **class_shuffle_strength** (Default: 0.5) - Shuffle intensity. 0-1.

#### Effects Parameters
- **contrast_strength** (Default: None) - Contrast effect strength. 0-1.
- **contrast_percussive** (Default: True) - React to percussive.
- **flash_strength** (Default: None) - Flash effect strength. 0-1.
- **flash_percussive** (Default: True) - React to percussive.
- **custom_effects** - List of EffectsGenerator objects.

#### Video Parameters
- **resolution** - Output resolution.
- **start** (Default: 0) - Start timestamp in seconds.
- **duration** - Duration in seconds. None = full audio.
- **fps** (Default: 30) - Frames per second.
- **output_audio** - Alternative audio for final video.
- **batch_size** (Default: 1) - Vectors per batch.
- **frame_batch_size** - Frames to process before writing to disk.

In [ ]:
from lucidsonicdreams import LucidSonicDream

# Full configuration example
L = LucidSonicDream(
    song=AUDIO_FILE,
    style=STYLES["abstract"],
    # Optionally use separate audio tracks for each component:
    # pulse_audio='drums.wav',
    # motion_audio='other.wav',
    # class_audio='melody.wav',
    # contrast_audio='bass.wav',
    # flash_audio='vocals.wav',
)

L.hallucinate(
    file_name='full_quality.mp4',
    
    # Video settings
    resolution=1024,
    fps=60,
    duration=None,  # Full song
    start=0,
    batch_size=1,
    frame_batch_size=3000,  # Write to disk every N frames
    
    # Initialization
    speed_fpm=6,
    truncation=1.0,
    
    # Motion settings
    motion_react=0.2,
    motion_randomness=0.01,
    motion_percussive=False,
    motion_harmonic=True,
    
    # Pulse settings
    pulse_react=0.15,
    pulse_percussive=True,
    pulse_harmonic=False,
    
    # Class settings (for models with classes like WikiArt)
    # classes=[1, 5, 9, 16, 23, 27, 28, 30, 50, 68, 71, 89],
    # dominant_classes_first=True,
    # class_pitch_react=0.2,
    class_smooth_seconds=2,
    # class_shuffle_seconds=8,
    # class_shuffle_strength=0.5,
    # class_complexity=1.0,
    
    # Effects
    # contrast_strength=0.5,
    # contrast_percussive=True,
    # flash_strength=0.3,
    # flash_percussive=True,
)

## 9. Batch Processing Multiple Songs

Process multiple songs with different styles automatically.

In [ ]:
import glob
import os
import shutil
import random
from lucidsonicdreams import LucidSonicDream

# Configuration
INPUT_FOLDER = "/content/drive/MyDrive/LucidSonicDreams/audio/"
OUTPUT_FOLDER = "/content/drive/MyDrive/LucidSonicDreams/outputs/"

# Find all audio files that don't have a corresponding .mp4
song_data = []
style_list = list(STYLES.values())

for file_type in [".mp3", ".wav"]:
    for audio_file in glob.glob(os.path.join(INPUT_FOLDER, f"*{file_type}")):
        video_file = audio_file.replace(file_type, '.mp4')
        if not os.path.exists(video_file):
            # Assign a random style or cycle through styles
            style = random.choice(style_list)
            song_data.append((audio_file, style))

print(f"Found {len(song_data)} songs to process:")
for song, style in song_data:
    print(f"  {os.path.basename(song)} -> {style}")

# Process each song
for audio_path, style in song_data:
    song_name = os.path.splitext(os.path.basename(audio_path))[0]
    output_name = f"{song_name}.mp4"
    
    print(f"\n{'='*50}")
    print(f"Processing: {song_name}")
    print(f"Style: {style}")
    print(f"{'='*50}")
    
    L = LucidSonicDream(audio_path, style=style)
    
    L.hallucinate(
        file_name=output_name,
        resolution=1024,
        fps=60,
        frame_batch_size=3000,
        truncation=1,
        duration=None,
        speed_fpm=6,
        batch_size=1,
        motion_percussive=False,
        motion_harmonic=True,
        motion_react=0.2,
        motion_randomness=0.01,
        pulse_percussive=True,
        pulse_harmonic=False,
        pulse_react=0.15,
        class_smooth_seconds=2,
    )
    
    # Move to output folder
    shutil.move(output_name, os.path.join(OUTPUT_FOLDER, output_name))
    print(f"Saved to: {OUTPUT_FOLDER}/{output_name}")

print("\nBatch processing complete!")

## 10. Custom Effects

Create your own reactive effects! The effects function must have:
- **array** - The image array to modify
- **strength** - Reactivity parameter (0-1)
- **amplitude** - Audio volume at current frame

Return a NumPy array representing the output image.

In [ ]:
import numpy as np
from scipy.ndimage import zoom
from lucidsonicdreams import EffectsGenerator, LucidSonicDream

def clipped_zoom(img, zoom_factor):
    """Zoom into image center without changing dimensions."""
    h, w = img.shape[:2]
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    if zoom_factor < 1:  # Zooming out
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple)
    elif zoom_factor > 1:  # Zooming in
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2
        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple)
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]
    else:
        out = img
    return out

def zoom_effect_func(array, strength, amplitude):
    """Zoom effect that reacts to audio."""
    zoom_amount = 1 + (0.05 * strength * amplitude)
    zoomed = clipped_zoom(array, zoom_amount)
    return zoomed.astype(np.uint8)

# Create the effect generator
zoom_effect = EffectsGenerator(
    func=zoom_effect_func,
    audio=AUDIO_FILE,
    strength=0.5,
    percussive=True  # React to beats
)

# Use with visualization
L = LucidSonicDream(AUDIO_FILE, style=STYLES["abstract"])

L.hallucinate(
    file_name='with_zoom_effect.mp4',
    resolution=512,
    fps=30,
    duration=30,
    custom_effects=[zoom_effect]
)

In [ ]:
# Swirl effect example
from skimage.transform import swirl

def swirl_effect_func(array, strength, amplitude):
    """Swirl effect that reacts to audio."""
    swirled = swirl(
        array,
        rotation=0,
        strength=100 * strength * amplitude,
        radius=650
    )
    return (swirled * 255).astype(np.uint8)

swirl_effect = EffectsGenerator(
    func=swirl_effect_func,
    audio=AUDIO_FILE,
    strength=0.2,
    percussive=False  # React to harmonic content
)

L = LucidSonicDream(AUDIO_FILE, style=STYLES["textures"])

L.hallucinate(
    file_name='with_swirl_effect.mp4',
    resolution=512,
    fps=30,
    duration=30,
    motion_react=0.15,
    speed_fpm=2,
    pulse_react=1.5,
    contrast_strength=1,
    flash_strength=1,
    custom_effects=[swirl_effect]
)

## 11. Custom Visualization Functions (Advanced)

Use any model that generates images from vectors! Example with BigGAN:

In [ ]:
# BigGAN example (uncomment to use)
# !pip install pytorch_pretrained_biggan

# from pytorch_pretrained_biggan import BigGAN, convert_to_images
# import torch

# biggan = BigGAN.from_pretrained('biggan-deep-512')
# biggan.to('cuda:0')

# def biggan_func(noise_batch, class_batch):
#     noise_tensor = torch.from_numpy(noise_batch).cuda()
#     class_tensor = torch.from_numpy(class_batch).cuda()
#     with torch.no_grad():
#         output_tensor = biggan(noise_tensor.float(), class_tensor.float(), truncation=1)
#     return convert_to_images(output_tensor.cpu())

# # ImageNet class labels: https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a
# L = LucidSonicDream(
#     AUDIO_FILE,
#     style=biggan_func,
#     input_shape=128,
#     num_possible_classes=1000
# )

# L.hallucinate(
#     file_name='biggan_output.mp4',
#     resolution=512,
#     duration=60,
#     speed_fpm=3,
#     classes=[13, 14, 22, 24, 301, 84, 99, 100, 134, 143, 393, 394],
#     class_shuffle_seconds=10,
#     class_shuffle_strength=0.1,
#     class_complexity=0.5,
#     class_smooth_seconds=4,
#     motion_react=0.35,
#     flash_strength=1,
#     contrast_strength=1
# )

## 12. Cleanup

In [ ]:
import torch
import gc

# Clear GPU memory
torch.cuda.empty_cache()
gc.collect()

# Clean temp files
!rm -rf /tmp/lsd_*.mp4 /tmp/lsd_*.wav
!rm -rf /content/stylegan2 /content/stylegan3

print("Cleanup complete!")
if torch.cuda.is_available():
    free, total = torch.cuda.mem_get_info()
    print(f"Free GPU memory: {free / 1024**2:.0f} MB")

## Tips & Troubleshooting

### Speed Optimizations (automatic)
- FP16 inference for StyleGAN (2x faster on GPU)
- BFloat16 inference for R3GAN (optimal precision)
- torch.compile() on PyTorch 2.0+
- torch.inference_mode()
- Pre-allocated arrays
- Vectorized NumPy operations

### Manual Tuning
- Lower resolution (512 vs 1024) for 4x faster generation
- Lower FPS (24 vs 60) for 2.5x faster
- Use `batch_size=2` if you have >12GB VRAM
- Use `frame_batch_size` to write frames to disk periodically (prevents OOM)

### Troubleshooting
- **OOM errors**: Reduce resolution or batch_size, add frame_batch_size
- **Slow generation**: Ensure GPU runtime is enabled
- **Codec errors**: Library auto-tries aac, libmp3lame, mp3 codecs
- **Style not found**: Use `show_styles()` or `show_r3gan_styles()` to see options

### Resources
- [Project GitHub](https://github.com/RayceRossum/lucid-sonic-dreams)
- [StyleGAN3 Models](https://github.com/NVlabs/stylegan3)
- [Pretrained StyleGAN Models](https://github.com/justinpinkney/awesome-pretrained-stylegan2)
- [R3GAN Paper](https://arxiv.org/abs/2501.05441) - "The GAN is dead; long live the GAN!"
- [R3GAN Models on HuggingFace](https://huggingface.co/brownvc)